# RPA-based reverse-engineering

In [ ]:
from collections import Counter
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from functools import partial, lru_cache
from scipy.stats import bernoulli
from concurrent.futures import ProcessPoolExecutor, as_completed

from IPython.display import HTML, display
from tqdm.auto import tqdm, trange
import tabulate

from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters, get_params
from pyecsca.ec.formula import FormulaAction
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod
from pyecsca.ec.mult import *
from pyecsca.misc.cfg import TemporaryConfig
from pyecsca.sca.trace.sampling import downsample_average, downsample_max
from pyecsca.sca.trace.process import normalize, rolling_mean
from pyecsca.sca.trace.combine import average, subtract
from pyecsca.sca.trace.test import welch_ttest
from pyecsca.sca.attack.leakage_model import HammingWeight, NormalNoice
from pyecsca.ec.context import DefaultContext, local
from pyecsca.sca.re.rpa import MultipleContext, rpa_distinguish
from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.plot import plot_trace, plot_traces

In [ ]:
%matplotlib inline
hv.extension("bokeh")

In [ ]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]  # The formulas are irrelevant for this method
dbl = coords.formulas["dbl-2007-bl"]
neg = coords.formulas["neg"]

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
a = Mod(0x4833d7aa73fa6694, p)
b = Mod(0xa6c44a61c5323f6a, p)
gx = Mod(0x5fd1f7d38d4f2333, p)
gy = Mod(0x21f43957d7e20ceb, p)
n = 0xc50de885003b80eb
h = 1

# A (0, y) RPA point on the above curve, in affine coords.
P0_aff = Point(coordsaff, x=Mod(0, p), y=Mod(0x1742befa24cd8a0d, p))

infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)

## Exploration
First select a bunch of multipliers. We will be trying to distinguish among these.

In [ ]:
multipliers = [
    LTRMultiplier(add, dbl, None, False, AccumulationOrder.PeqPR, True, True),
    LTRMultiplier(add, dbl, None, True, AccumulationOrder.PeqPR, True, True),
    RTLMultiplier(add, dbl, None, False, AccumulationOrder.PeqPR, True),
    RTLMultiplier(add, dbl, None, True, AccumulationOrder.PeqPR, False),
    SimpleLadderMultiplier(add, dbl, None, True, True),
    BinaryNAFMultiplier(add, dbl, neg, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    WindowNAFMultiplier(add, dbl, neg, 3, None, AccumulationOrder.PeqPR, True, True),
    WindowNAFMultiplier(add, dbl, neg, 4, None, AccumulationOrder.PeqPR, True, True),
    WindowNAFMultiplier(add, dbl, neg, 5, None, AccumulationOrder.PeqPR, True, True),
    #WindowNAFMultiplier(add, dbl, neg, 4, None, AccumulationOrder.PeqPR, False, True),  # Same set of multiples as Window NAF with precompute_negation
    SlidingWindowMultiplier(add, dbl, 3, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    SlidingWindowMultiplier(add, dbl, 4, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    SlidingWindowMultiplier(add, dbl, 5, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    FixedWindowLTRMultiplier(add, dbl, 4, None, AccumulationOrder.PeqPR, True),
    FixedWindowLTRMultiplier(add, dbl, 5, None, AccumulationOrder.PeqPR, True),
    FixedWindowLTRMultiplier(add, dbl, 8, None, AccumulationOrder.PeqPR, True),
    FullPrecompMultiplier(add, dbl, None, True, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True, True),
    FullPrecompMultiplier(add, dbl, None, False, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True, True),
    #FullPrecompMultiplier(add, dbl, None, False, ProcessingDirection.RTL, AccumulationOrder.PeqPR, True, True),  # Same set of multiples as RTL multiplier
    BGMWMultiplier(add, dbl, 2, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    BGMWMultiplier(add, dbl, 3, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    BGMWMultiplier(add, dbl, 5, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    CombMultiplier(add, dbl, 2, None, AccumulationOrder.PeqPR, True),
    CombMultiplier(add, dbl, 3, None, AccumulationOrder.PeqPR, True),
    CombMultiplier(add, dbl, 5, None, AccumulationOrder.PeqPR, True)
]

Then select a random scalar and simulate computation using all of the multipliers, track the multiples, print the projective and affine results.

In [ ]:
scalar = 0b1000000000000000000000000000000000000000000000000
scalar = 0b1111111111111111111111111111111111111111111111111
scalar = 0b1010101010101010101010101010101010101010101010101
scalar = 0b1111111111111111111111110000000000000000000000000
scalar = 123456789123456789
scarar = 8750920244948492046
# multiples is a mapping from a multiple (integer) to a set of scalar multipliers that compute said multiple when doing [scalar]P
multiples = {}

table = [["Multiplier", "multiples"]]

for mult in multipliers:
    with local(MultipleContext()) as ctx:
        mult.init(params, g)
        res = mult.multiply(scalar)
    for m in ctx.points.values():
        s = multiples.setdefault(m, set())
        s.add(mult)
    table.append([str(mult), str(list(ctx.points.values()))])

display(HTML(tabulate.tabulate(table, tablefmt="html", headers="firstrow")))

Pick a multiple `k` that is computed by some multiplier for the scalar,
invert it mod n, and do `[k^-1]P0` to obtain a point `P0_target`,
such that, `[k]P0_target = P0` and `P0` has a zero coordinate.

In [ ]:
k = 108
kinv = Mod(k, n).inverse()
P0_target = curve.affine_multiply(P0_aff, int(kinv)).to_model(coords, curve)

print("Original P0", P0_aff)
print("P0_target  ", P0_target.to_affine())
print("Verify P0  ", curve.affine_multiply(P0_target.to_affine(), k))

Now go over the multipliers with P0_target and the original scalar as input.
Then look whether a zero coordinate point was computed.
Also look at whether the multiple "k" was computed. These two should be the same.

In [ ]:
table = [["Multiplier", "zero present", "multiple computed"]]

for mult in multipliers:
    with local(MultipleContext()) as ctx:
        mult.init(params, P0_target)
        res = mult.multiply(scalar)
    zero = any(map(lambda P: P.X == 0 or P.Y == 0, ctx.points.keys()))
    multiple = k in ctx.points.values()
    table.append([str(mult), f"<b>{zero}</b>" if zero else zero, f"<b>{multiple}</b>" if multiple else multiple])

display(HTML(tabulate.tabulate(table, tablefmt="unsafehtml", headers="firstrow", colalign=("left", "center", "center"))))

Now lets look at the relation of multiples to multipliers.

In [ ]:
table = [["Multiple", "Multipliers"]]
for multiple, mults in multiples.items():
    table.append([bin(multiple), [mult.__class__.__name__ for mult in mults]])

display(HTML(tabulate.tabulate(table, tablefmt="html", headers="firstrow")))

Note that all of the exploration so far was in a context of a fixed scalar. Even though for a given scalar some multipliers might be indistinguishable from the perspective of the multiples they compute, there may be other scalars that distinguish them.

## Reverse-engineering

### Oracle simulation
The `simulated_oracle` function simulates an RPA oracle that detect a zero coordinate point in the scalar multiplication.
This can be used by the `rpa_distinguish` function to distinguish the true scalar multiplier. The oracle is parametrized with the simulated multiplier index in the table of multipliers (it simulates this "real" multiplier). Furthermore, lets also examine a `noisy_oracle` (with a flip probability) and a `biased_oracle` (with asymmetric flip probability).

Note that the oracle has two additional parameters `measure_init` and `measure_multiply` which determine whether the oracle considers the zero coordinate point in scalar multiplier initialization (precomputation) and in scalar multiplier multiplication, respectively. This is important for scalar multipliers with precomputation as there one might be able to separate the precomputation and multiplication stages and obtain oracle answers on both separately.

In [ ]:
def simulated_oracle(scalar, affine_point, simulate_mult_id=0, measure_init=True, measure_multiply=True, randomize=False):
    real_mult = multipliers[simulate_mult_id]
    point = affine_point.to_model(params.curve.coordinate_model, params.curve, randomized=randomize)
    
    # Simulate the multiplier init
    with local(MultipleContext()) as ctx:
        real_mult.init(params, point)
    init_points = set(ctx.parents.keys())
    init_parents = set(sum((ctx.parents[point] for point in init_points), []))
    # Did zero happen in some input point during the init?
    init_zero = any(map(lambda P: P.X == 0 or P.Y == 0, init_parents))
    
    # Simulate the multiplier multiply
    with local(ctx) as ctx:
        real_mult.multiply(scalar)
    all_points = set(ctx.parents.keys())
    multiply_parents = set(sum((ctx.parents[point] for point in all_points - init_points), []))
    # Did zero happen in some input point during the multiply?
    multiply_zero = any(map(lambda P: P.X == 0 or P.Y == 0, multiply_parents))
    real_result = (init_zero and measure_init) or (multiply_zero and measure_multiply)
    return real_result

def noisy_oracle(oracle, flip_proba=0):
    def noisy(*args, **kwargs):
        real_result = oracle(*args, **kwargs)
        change = bernoulli(flip_proba).rvs()
        return bool(real_result ^ change)
    return noisy

def biased_oracle(oracle, flip_0=0, flip_1=0):
    def biased(*args, **kwargs):
        real_result = oracle(*args, **kwargs)
        change = bernoulli(flip_1).rvs() if real_result else bernoulli(flip_0).rvs()
        return bool(real_result ^ change)
    return biased

We can see how the RPA-RE method distinguishes a given multiplier:

In [ ]:
p256 = get_params("secg", "secp256r1", "projective")
res = rpa_distinguish(params, multipliers, simulated_oracle)

Now we can examine how the method performs in the presence of noise and with various majority vote parameters. Note that the code below spawns several processes (`num_cores`) and saturates their CPU fully, so set this to something appropriate.

In [ ]:
errs = (0, 0.1, 0.2, 0.3, 0.4, 0.5)
majs = (1, 3, 5, 7, 9, 11)
correct_tries = np.zeros((len(errs), len(majs)))
precise_tries = np.zeros((len(errs), len(majs)))
query_tries = np.zeros((len(errs), len(majs)))
total_tries = 0

num_tries = 50

In [ ]:
num_cores = 30

def measure_mult(params, multipliers, simulated_oracle, i, mult, err, majority):
    correct = 0
    precise = 0
    calls = 0
    p = lru_cache(maxsize=2)(partial(simulated_oracle, simulate_mult_id=i))
    noisy = noisy_oracle(p, flip_proba=err)
    def oracle(scalar, affine_point):
        nonlocal calls
        calls += 1
        return noisy(scalar, affine_point)
    for j in range(num_tries):
        res = rpa_distinguish(params, multipliers, oracle, majority=majority)
        if mult in res:
            correct += 1
            if len(res) == 1:
                precise += 1
    return correct, precise, calls

with TemporaryConfig() as cfg:
    cfg.log.enabled = False
    with ProcessPoolExecutor(max_workers=num_cores) as pool:
        futures = []
        args = []
        for i, mult in enumerate(multipliers):
            for err in errs:
                for majority in majs:
                    a = (params, multipliers, simulated_oracle, i, mult, err, majority)
                    futures.append(pool.submit(measure_mult, *a))
                    args.append(a)
        results = [None for _ in futures]
        for future in tqdm(as_completed(futures), total=len(futures)):
            j = futures.index(future)
            a = args[j]
            results[j] = future.result()

Now we accumulate the results across the error rate and majority vote parameters.

In [ ]:
for a, result in zip(args, results):
    i = errs.index(a[5])
    j = len(majs) - majs.index(a[6]) - 1
    correct_tries[i, j] += result[0]
    precise_tries[i, j] += result[1]
    query_tries[i, j] += result[2]
total_tries += num_tries

correct_rate = (correct_tries * 100) / (total_tries * len(multipliers))
precise_rate = (precise_tries * 100) / (total_tries * len(multipliers))
query_rate = query_tries / (total_tries * len(multipliers))

We can plot two heatmaps:
 - One for the average number of queries to the oracle.
 - One for the success rate of the distinguisher.

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(query_rate.T, cmap="plasma")
cbar_ax = fig.add_axes((0.90, 0.15, 0.04, 0.69))
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.ax.set_ylabel("Average oracle query rate", rotation=-90, va="bottom")

ax.set_xticks(np.arange(len(errs)), labels=errs)
ax.set_yticks(np.arange(len(majs)), labels=reversed(majs))
ax.set_xlabel("err")
ax.set_ylabel("majority")
for i in range(len(errs)):
    for j in range(len(majs)):
        text = ax.text(i, j, f"{query_rate[i, j]:.2f}",
                       ha="center", va="center", color="w")
fig.savefig("rpa_re_query_rate.png", bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(correct_rate.T, vmin=0, cmap="viridis")
cbar_ax = fig.add_axes((0.90, 0.15, 0.04, 0.69))
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.ax.set_ylabel("Success rate", rotation=-90, va="bottom")
cbar.ax.axhline(100 / len(multipliers), color="red", linestyle="--")

ax.set_xticks(np.arange(len(errs)), labels=errs)
ax.set_yticks(np.arange(len(majs)), labels=reversed(majs))
ax.set_xlabel("err")
ax.set_ylabel("majority")
for i in range(len(errs)):
    for j in range(len(majs)):
        text = ax.text(i, j, f"{correct_rate[i, j]:.2f}%",
                       ha="center", va="center", color="w")
fig.savefig("rpa_re_success_rate.png", bbox_inches="tight")
plt.show()

### Method simulation

The `simulate_trace` function simulates a Hamming weight leakage trace of a given multiplier computing a scalar multiple.
This is used by the `simulated_rpa_trace` function that does the RPA attack on simulated traces and returns the differential
trace. This is in turn used to build the `simulated_rpa_oracle` which can be used by the `rpa_distinguish` function to perform
RPA-RE and distinguish the true scalar multiplier. The oracle is parametrized with the simulated multiplier index in the table of multipliers (it simulates this "real" multiplier).

In [ ]:
def simulate_trace(mult, scalar, point):
    with local(DefaultContext()) as ctx:
        mult.init(params, point)
        mult.multiply(scalar)

    lm = HammingWeight()
    trace = []

    def callback(action):
        if isinstance(action, FormulaAction):
            for intermediate in action.op_results:
                leak = lm(intermediate.value)
                trace.append(leak)

    ctx.actions.walk(callback)
    return Trace(np.array(trace))

def simulated_rpa_trace(mult, scalar, affine_point, noise):
    target_point = affine_point.to_model(params.curve.coordinate_model, params.curve)
    random_point = params.curve.affine_random().to_model(params.curve.coordinate_model, params.curve)

    random_traces = [noise(simulate_trace(mult, scalar, random_point)) for _ in range(10)]
    target_traces = [noise(simulate_trace(mult, scalar, target_point)) for _ in range(500)]

    random_avg = average(*random_traces)
    target_avg = average(*target_traces)

    diff_trace = downsample_max(subtract(random_avg, target_avg), 25)
    return diff_trace

def simulated_rpa_oracle(scalar, affine_point, simulate_mult_id = 0):
    real_mult = multipliers[simulate_mult_id]
    noise = NormalNoice(0, 1)
    diff_trace = normalize(simulated_rpa_trace(real_mult, scalar, affine_point, noise))
    peaks, props = find_peaks(diff_trace.samples, height=4)
    return len(peaks) != 0

In [ ]:
table = [["True multiplier", "Reversed", "Correct", "Remaining"]]
with TemporaryConfig() as cfg:
    cfg.log.enabled = False
    for i, mult in tqdm(enumerate(multipliers)):
        res = rpa_distinguish(params, multipliers, partial(simulated_rpa_oracle, simulate_mult_id = i))
        table.append([mult, res, mult in res, len(res)])
display(HTML(tabulate.tabulate(table, tablefmt="html", headers="firstrow")))

Note that the oracle function above has several parameters, like noise amplitude, amount of traces simulated, amount of downsampling and peak finding height threshold. The cell below compares the differential RPA trace when the multiple is computed in the simulation vs when it is not.

In [ ]:
diff_real = normalize(simulated_rpa_trace(multipliers[0], scalar, P0_target.to_affine(), NormalNoice(0, 1)))
diff_nothing = normalize(simulated_rpa_trace(multipliers[5], scalar, P0_target.to_affine(), NormalNoice(0, 1)))
plot_traces(diff_real, diff_nothing).opts(responsive=True, height=600)